# Exercise 2.6: Creating Dashboards with Python

## 1. Install libraries, set system path, import dataset

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime as dt
from streamlit_keplergl import keplergl_static

In [2]:
# Set system path

path = r'C:\Users\Stony\OneDrive\CareerFoundry\Data Specialization Course\envs\Citi-Bike'

In [3]:
# Import dataset from prior exercise

df_final = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'final_data.csv'), index_col = 0)

C:\Users\Stony\AppData\Local\Temp\ipykernel_23040\1699888674.py:3: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'final_data.csv'), index_col = 0)


In [4]:
df_final.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
avgTemp               float64
bike_rides_daily        int64
started                object
ride_date              object
dtype: object

## 2. Create a month column

In [5]:
# Create a month column from started_at

df_final['date'] = pd.to_datetime(df_final['started_at'])
df_final['month'] = df_final['date'].dt.month
df_final['month'] = df_final['month'].astype('int')

In [6]:
# Create a season column

df_final['season'] = [
"winter" if (month == 12 or 1 <= month <= 2)
    else "spring" if (3 < month <= 5)
    else "summer" if (6 <= month <= 8)
    else "fall"
for month in df_final['month']
    ]

## 3. Create a bar chart in Plotly

In [7]:
# Create a dataframe of the top 20 stations in NYC

df_final['value'] = 1
df_final_groupby_bar = df_final.groupby('start_station_name', as_index=False).agg({'value' : 'sum'})
top20 = df_final_groupby_bar.nlargest(20, 'value')

In [8]:
# Create a bar chart in Plotly of the 20 busiest stations in NYC

fig = go.Figure(go.Bar(x = top20['start_station_name'], y = top20['value']))
fig.show()

In [9]:
# Change the color of the bar plots to Blue based on the value of the stations

fig = go.Figure(go.Bar(x = top20['start_station_name'], y = top20['value'], marker={'color': top20['value'],'colorscale': 'Blugrn'}))
fig.show()

In [10]:
# Update the layout of the bar chart plot by adding a plot title, axis titles, and adjusting the size of the plot

fig.update_layout(
    title = 'Top 20 most popular bike stations in New York City',
    xaxis_title = 'Start stations',
    yaxis_title ='Sum of trips',
    width = 900, height = 600
)

## 4. Create a dual-axis line chart in Plotly

In [11]:
# Create a dataframe with the columns required for the dual-axis plot: date (started_at), total bike rides, and average temperature

df_dual = df_final[['started_at', 'bike_rides_daily', 'avgTemp']]

In [12]:
df_dual.shape
df_dual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29838166 entries, 2022-01-01 to 2022-12-31
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   started_at        object 
 1   bike_rides_daily  int64  
 2   avgTemp           float64
dtypes: float64(1), int64(1), object(1)
memory usage: 910.6+ MB


In [13]:
# Step 1: Convert the date column to datetime

df_dual['started_at'] = pd.to_datetime(df_dual['started_at'])

# Step 2: Extract the date part from the datetime

df_dual['ride_date'] = df_dual['started_at'].dt.date

# Step 3: Group by date and count the rides

daily_rides = df_dual.groupby('ride_date').agg(total_rides=('bike_rides_daily', 'count'), average_temp=('avgTemp', 'mean')).reset_index()

# Create a complete date range for the year

date_range = pd.date_range(start='2022-01-01', end='2022-12-31')

# Reindex the daily_rides DataFrame to include all dates in the range

daily_rides = daily_rides.set_index('ride_date').reindex(date_range, fill_value=0).reset_index()

# Rename the columns for clarity

daily_rides.columns = ['date', 'total_rides', 'avg_daily_temp']

C:\Users\Stony\AppData\Local\Temp\ipykernel_23040\3343487931.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Stony\AppData\Local\Temp\ipykernel_23040\3343487931.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
# Export final dataframe

daily_rides.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'daily_rides_per_day.csv'))


In [15]:
print(daily_rides)

          date  total_rides  avg_daily_temp
0   2022-01-01        20428            11.6
1   2022-01-02        43009            11.4
2   2022-01-03        33189             1.4
3   2022-01-04        36842            -2.7
4   2022-01-05        34230             3.2
..         ...          ...             ...
360 2022-12-27        33044            -0.7
361 2022-12-28        44027             3.4
362 2022-12-29        51674             6.4
363 2022-12-30        62298             9.3
364 2022-12-31        31466             8.2

[365 rows x 3 columns]


In [17]:
# Line chart

fig = make_subplots(specs = [[{"secondary_y": True}]])

fig.add_trace(
go.Scatter(x = daily_rides['date'], y = daily_rides['total_rides'], name = 'Daily bike rides', marker={'color': daily_rides['total_rides'],'color': 'blue'}),
secondary_y = False
)

fig.add_trace(
go.Scatter(x=daily_rides['date'], y = daily_rides['avg_daily_temp'], name = 'Daily temperature (°C)', marker={'color': daily_rides['avg_daily_temp'],'color': 'red'}),
secondary_y=True
)